# Relatorio TP2
  ## Feito por: 
    - Artur Paredes     A46347
    - Duarte Domingues A45140
    - Pedro Henriques  A45415
    


## Introdução
No segundo trabalho prático da disciplina CSM iremos abordar os conceitos de compressão de dados sem perdas baseados na teoria da informação, especialmente a codificação de Huffman.


## Código Huffman


- A codificação de Huffman é um método de Método de codificação entrópica que utiliza as probabilidades de ocorrência de símbolos no conjunto de dados a ser comprimido para determinar códigos de comprimento variável.

- O output deste algoritmo é uma tabela de códigos para codificar um símbolo de fonte. Símbolos com maior ocorrência são representados com um menor número de bits que símbolos com ocorrência menor.

### Algoritmo

- 1. Colocar os símbolos numa lista por ordem decrescente de probabilidade 
- 2. Escolher os dois simbolos com menor frequência e agrupá-los num novo símbolo com probabilidade igual à soma destes dois. 
- 3. Inserir o novo símbolo na lista ordenada.
- 4. Voltar ao ponto 2 até que cada lista tenha apenas um símbolo.
- 5. Atribuir códigos partindo do ultimo símbolo criado.



    - Neste exercicio foi nos pedido a realização de um gerador de código de Huffman. 
    A codificação de huffman consiste em utilizar uma lista de simbolos e respetivas probabilidades, para através da junção de simbolos, sermos capaz de comprimir, pelo menos em casos normais, uma mensagem.

    

In [1]:
import numpy as np
import copy
import cv2
import matplotlib.pyplot as plt
import time, re
from bitstring import BitArray
from os import path

### Ex1 : Gerar tabelas de huffman

    - Neste exercicio vamos explicar como procedemos de modo a implementar o algoritmo que origina o código de Huffman
    - Tendo em conta um caso base, com apenas dois simbolos (A e B) iriamos atribuir os valores lógicos 0 e 1 respetivamente
    - Ao adicionar mais elementos, juntar os elementos com menor probabilidade sucessivamente até termos apenas 2 elementos.
    - Ao ter apenas 2 elementos, chegamos ás folhas da nossa árvore, atribuindo um valor logico a cada folha
    - Assim todos os outros elementos têm um código binario que corresponde á sequencia dos caracteres das folhas e ramos diretamente a cima deles.
    
    De modo a auxiliar á realização deste metodo, foram utilizados alguns metodos, cujos comentarios explicam sucintamente qual o seu objetivo.

In [2]:
def gen_huff_table(word_dic= {}, symbols=[], probs=[],):
    if(len(word_dic)!= 0):
        return gen_huff_table_from_dic(word_dic)
    elif (len(symbols)==len(probs) and len(symbols)!=0):
        return gen_huff_table_from_dic(dict(zip(symbols, probs)))
    else:
        raise ValueError('''Argumentos invalidos, forneca uma tabela com chave simbolo e valor 
                         probabilidade ou simbolos e probabilidades em listas distintas de igual valor''')
        
def gen_huff_table_from_dic(word_dic):
    dic_bits = {}

    for i in list(word_dic.keys()):
        dic_bits[i] = ''

    dic_list = []

    dic_list.append(sort_dic(word_dic))

    #Cria uma arvore de letras e atribui o código a cada símbolo
    while (len(dic_list[-1])!=1):
        previous_dic = dic_list[-1]

        keys = list(previous_dic.keys())
        key1 = keys[-1]
        val1 = previous_dic[key1]
        key2 = keys[-2]
        val2 = previous_dic[key2]

        for i in key1:
            dic_bits[i] = '1' + dic_bits[i]

        for i in key2:
            dic_bits[i] = '0' + dic_bits[i]


        newKey = key2 + key1
        newValue = val1 + val2

        new_dic = copy.deepcopy(previous_dic)
        new_dic.pop(key1)
        new_dic.pop(key2)
        new_dic[newKey] = newValue

        dic_list.append(sort_dic(new_dic))

    return dic_bits

def sort_dic(dic):
    dic_output = {}
    sorted_keys = sorted(dic, key=dic.get, reverse=True)
    for key in sorted_keys:
        dic_output[key] = dic[key]
    return dic_output

def convert(message):
    '''Convert a mensagem para char, desde que esta seja iteravel (strings, arrays, tuplos, etc...)'''
    return list(map(str, message))
    

def getProbDic(message):
    '''Retorna um dicionário com as probabilidades de cada símbolo'''
    message = convert(message)
    unique, counts = np.unique(message, return_counts=True)
    counts = counts/len(message)
    dic = dict(zip(unique, counts))

    return dic


s = ['a', 'b', 'c','d','e']
p = [.4, .3, .1, .05, .15]

tabela1 = gen_huff_table(symbols=s, probs=p)
print(f'teste 1- gerar com simbolos e probabilidades\n simbolos - {s} \n probabilidades - {p} \n resultado \n {tabela1}')

m2 = '45415'
tabela2 = gen_huff_table(getProbDic(m2))
print(f'teste 1- gerar com mensagem \n mensagem {m2} \n resultado \n {tabela2}')

teste 1- gerar com simbolos e probabilidades
 simbolos - ['a', 'b', 'c', 'd', 'e'] 
 probabilidades - [0.4, 0.3, 0.1, 0.05, 0.15] 
 resultado 
 {'a': '1', 'b': '01', 'c': '0000', 'd': '0001', 'e': '001'}
teste 1- gerar com mensagem 
 mensagem 45415 
 resultado 
 {'1': '01', '4': '1', '5': '00'}


## Exercício 2 


- Neste exercício elaborámos uma função denominada,  “encode_huff”, esta função recebe dois argumentos, uma mensagem e a tabela de códigos,resultando uma sequência de bits com a mensagem codificada.

- Foram elaborados testes ao funcionamentos da função com as mensagens ("aaaabbccdee") e ("45415"), tendo sido obtido valores corretos para a sequência de bits gerada.


In [3]:
def encode_huff (message , symbol_code_table):
    message = convert(message)
    output = ''
    for i in message:
        output += symbol_code_table[i]
    return output


m1 = 'aaaabbccdee'
code1 = encode_huff(m1, tabela1)
print(f'teste á encodificacao - mensagem {m1}\n tabela simbolos {tabela1} \n resultado = {code1}')
code2 = encode_huff(m2, tabela2)
print(f'teste á encodificacao - mensagem {m2} \n tabela simbolos {tabela2}\n resultado = {code2}')

teste á encodificacao - mensagem aaaabbccdee
 tabela simbolos {'a': '1', 'b': '01', 'c': '0000', 'd': '0001', 'e': '001'} 
 resultado = 11110101000000000001001001
teste á encodificacao - mensagem 45415 
 tabela simbolos {'1': '01', '4': '1', '5': '00'}
 resultado = 10010100


## Exercício 3

### Decodificação de Huffman

Neste exercício elaborámos uma função para realizar a descodificação da sequencia de bits obtida no exercício anterior.

- Começamos por criar duas Strings, uma String “mensagem” que irá conter a mensagem de output da descodificação da sequência de códigos. Outra String “bin_code” que irá ser usada para percorrer os valores da sequencia de bits, para depois poder ser comparada com os códigos.

- Como num codigo de Huffman, nenhum codigo é prefixo de outro, lemos os bits  da sequência binária de código para o “bin_code”, caso seja encontrado um código existente o seu símbolo respetivo é adicionado à mensagem de output e a String “bin_code” é esvaziada, sucessivamente até ser realizada a descodificação, caso contrario continuamos a juntar bits á string "bin_code" até que seja possivel verificar a condição anterior.

In [5]:
def decode_huff (binary_codes, symbol_code_table):   
    bin_code = ''
    mensagem = ''
    for bit in binary_codes:
        bin_code += bit
        
        for simbolo, codigo in symbol_code_table.items():
            if(codigo == bin_code):
                mensagem+=simbolo
                bin_code = ''
                break

    return mensagem

r1 = decode_huff(code1, tabela1)
print(f'teste á descodificacao - mensagem original "{m1}"\n tabela simbolos {tabela1}, codigos binarios {code1} \n  resultado = {r1}')
r2 = decode_huff(code2, tabela2)
print(f'teste á descodificacao - mensagem original "{m2}"\n tabela simbolos {tabela2}, codigos binarios {code2} \n  resultado = {r2}')

teste á descodificacao - mensagem original "aaaabbccdee"
 tabela simbolos {'a': '1', 'b': '01', 'c': '0000', 'd': '0001', 'e': '001'}, codigos binarios 11110101000000000001001001 
  resultado = aaaabbccdee
teste á descodificacao - mensagem original "45415"
 tabela simbolos {'1': '01', '4': '1', '5': '00'}, codigos binarios 10010100 
  resultado = 45415


## Exercício 4

### Escrever para o ficheiro

Neste exercicio elaborámos uma função "writeArray2File" que permite a escrita da mensagem codificada para um ficheiro. Tendo sido necessário também a realização duma função denominada "dictToBinary" que permite através do dicionário obter uma sequência de bits.

A função dictToBinary percorre as chaves e valores do dicionário, guardando os seus caracteres ascii correspondentes, sendo que as chaves são guardadas com 8 bits e 5 bits de header do simbolo, que indica o comprimento do valor que se segue.




In [85]:
def writeArray2File(filename, message, dic):
    with(open(filename, 'wb')) as file:
        dicBin = dictToBinary(dic)
        final_out = dicBin + message
        print('so com dic e message ', final_out)
        
        #escrever no inicio o numero de bits adicionados, depois a mensagem , depois o padding
        #Caso a string ja tenha um numero adequado para ser formatada em bytes, 
        add_bits = 8 - ((len(final_out) + 3) % 8)
        add_bits = add_bits if add_bits!=8 else 0

        len_bits = '{0:03b}'.format(add_bits)
        final_out = len_bits + final_out +  '0'*add_bits 
        
        bytes_array = re.findall('[01]{8}',final_out)
        
        for byte in bytes_array:
            byte = int(byte,2).to_bytes(1,'little')
            file.write(byte)
        
        file.close()
        return True, final_out
    return False

def dictToBinary(dic):
    final_out = ''
    
    #primeiro bit guarda o key_type
    #guardar em binario o comprimento do dicionario
    len_bits = '{0:08b}'.format(len(dic))
    final_out += len_bits

    for key, value in dic.items():
        bitsKey = ''
        
        #guarda sempre o caracter ascii correspondente
        bitsKey = '{0:08b}'.format(int(ord(key)))
        bitLenVal = '{0:05b}'.format(len(value))

        final_out += bitsKey + bitLenVal  # 8 bits de chae, 5 bits de header do simbolo
        final_out += value
    
    #numero de bits por par chave-valor == 8+5+N
    return final_out

dicionarioBinario1 = dictToBinary(tabela1)
print(f'teste a dictToBinary key_type = "char"\n  dicionario original {tabela1}\n  codificacao em binario {dicionarioBinario1}')

dicionarioBinario2 = dictToBinary(tabela2)
print(f'teste a dictToBinary key_type = "int"\n  dicionario original {tabela2}\n  codificacao em binario {dicionarioBinario2}')


sucesso, trama_escrita = writeArray2File('ficheiro.bin',  code2,tabela2, )
print(f'teste a writeArray2File sucesso: {s}\n\tdicionario em binario {dicionarioBinario2}\n\tmensagem binaria {code2}\n\ttrama escrita {trama_escrita}')


teste a dictToBinary key_type = "char"
  dicionario original {'a': '1', 'b': '01', 'c': '0000', 'd': '0001', 'e': '001'}
  codificacao em binario 000001010110000100001101100010000100101100011001000000011001000010000010110010100011001
teste a dictToBinary key_type = "int"
  dicionario original {'1': '01', '4': '1', '5': '00'}
  codificacao em binario 0000001100110001000100100110100000011001101010001000
so com dic e message  000000110011000100010010011010000001100110101000100010010100
teste a writeArray2File sucesso: ['a', 'b', 'c', 'd', 'e']
	dicionario em binario 0000001100110001000100100110100000011001101010001000
	mensagem binaria 10010100
	trama escrita 0010000001100110001000100100110100000011001101010001000100101000


## Exercício 5

### Ler do ficheiro

Neste exercicio foi realziada uma função ("readFile2Array") que lê uma sequência de bits contida numf ficheiro consoante o nome do ficheiro. Teve que ser elaborada também uma função denominada ("getDicAndMensageFromBin") para poder obter a mensagem através de uma trama de bits.



In [8]:
def readFile2Array(filename):
    f = open(filename, 'rb')
    f = f.read()
    a = BitArray(f)
    a = a.bin

    print("Tamanho da trama lida do ficheiro ", len(a))

    #Remove os bits adicionados no final da trama
    added_bits = [0, 0, 0, 0, 0]
    for i in range(3):
        added_bits.append(int(a[i]))
    added_bits = np.packbits(added_bits)

    a = a[:-int(added_bits)]

    #Lê os primeiros 8 bits que diz o número de símbolos
    symbol_size = []
    for i in range(3, 11):
        symbol_size.append(int(a[i]))
    symbol_size = np.packbits(symbol_size)

    dic_decoded = {}

    #Começa no indice 11 porque os 11 anteriores já foram lidos em cima
    idx = 11
    while(len(dic_decoded) < symbol_size[0]):

        #Lê 8 bits da key
        key = []
        for i in range(8):
            key.append(int(a[idx]))
            idx += 1
        key = chr(np.packbits(key)[0])

        #Lê os 5 bits que dizem o comprimento do valor associado à key de cima
        #A array começa com 3 bits a 0 porque o packbits só aceita 8 bits (3 + 5)
        value_size = [0, 0, 0]
        for i in range(5):
            value_size.append(int(a[idx]))
            idx += 1
        value_size = np.packbits(value_size)

        #Guarda a trama de bits correspondentes ao "value", no formato
        # "10101" como estava no dicionário original
        value = ""
        for i in range(value_size[0]):
            value += a[idx]
            idx += 1

        #Constroi o dicionário
        dic_decoded[key] = value

    #Controi a mensagem
    mensagem = ""
    while(idx < len(a)):
        mensagem += a[idx]
        idx+=1

    return dic_decoded, mensagem

dic_decoded , mensagem_lida = readFile2Array('ficheiro.bin')
print(f'teste a readfile2Array:\n\tmensagem esperada "{code2}" \t dicionario esperado "{tabela2}"\n\tmensagem lida {mensagem_lida} \n\tdicionario lido {dic_decoded}')


Tamanho da trama lida do ficheiro  64
teste a readfile2Array:
	mensagem esperada "10010100" 	 dicionario esperado "{'1': '01', '4': '1', '5': '00'}"
	mensagem lida 10010100 
	dicionario lido {'1': '01', '4': '1', '5': '00'}


## Exercício 6

Neste exercício testámos as funções realizada ao longo do trabalho prático na imagem ("LenaGray.tif"). Realizámos as seguintes etapas:

- 1 Converter a imagem num array através do método np.ravel.
- 2 Gerar a tabela de codificação.
- 3 Medição da  entropia e o número médio de bits por símbolo, sendo calculada a  eficiência.
- 4 Realização da  codificação da mensagem contida no ficheiro, sendo realizado o tempo que demora.
- 5 Gravação e leitura da mensagem em ficheiro.
- 6 Ler do ficheiro o conjunto de bits.
- 7 Realizar a descodificação da mensagem.
- 8 Comparação da mensagem descodificada com a original.

In [10]:
def messageToImage(imgname, message_decoded):
    array = np.zeros([512,512,3])
    a = 0
    for i in range(len(array)):
        for j in range(len(array[i])):
            array[i][j] = ord(message_decoded[a])
            a += 1

    cv2.imwrite(imgname, array)

def ex6():
    
    #import matplotlib.pyplot as plt
        
    filename = "File.txt"
    imgname = "LenaTP2.jpg"


    # Lê a imagem em níveis de cinzento
    imgpath = "lena.tiff"
    img_read = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)

    # Converte a imagem (matriz) numa sequência de números (array)
    img_array = np.ravel(img_read)
    img_array = convert(img_array)


    #Alínea a)
    t0 = time.time()
    message_prob = getProbDic(img_array)
    symbol_code_table = gen_huff_table(word_dic=message_prob)
    t1 = time.time()
    print ("Huff table time: ", t1-t0)   

    #Alínea b)
    probs = np.fromiter(getProbDic(img_array).values(), float)
        
    Entropy = -1.* np.sum(probs * np.log2(probs))
    print(f'Entropia {Entropy}')
    
    L = sum([len(v) for v in symbol_code_table.values()])/len(symbol_code_table)
    print(f'Num medio bits {L}')
    
    
    Eficiency = Entropy/L
    print(f'Eficiencia do codigo {Eficiency}')
    
    #Alínea c)
    t0 = time.time()
    message_encoded = encode_huff(img_array, symbol_code_table)
    t1 = time.time()
    print("Encode message time: ", t1-t0)

    #Alínea d)
    #Escreve a mensagem em binário para um ficheiro
    writeArray2File(filename, symbol_code_table, message_encoded, "char")


    #Alínea e)
    #Lê a mensagem do ficheiro e retorna-a numa string
    t0 = time.time()
    dic_decoded, message_decoded = readFile2Array(filename)
    t1 = time.time()
    print("Decoded message and dictionary time: ", t1-t0)

    #Alínea f)
    #Descodifica a mensagem
    t0 = time.time()
    message_decoded_ints = decode_huff(message_decoded, dic_decoded)
    t1 = time.time()
    print("Decoded message to ints time: ", t1-t0)


    #Alínea g)
    #Deteção de erro
    #Testa se a mensagem enviada (array de símbolos)
    #é igual à mensagem recebida (descodificada)
    print("Mensagem codificada == Mensagem descodificada: ", message_decoded == message_encoded)

    #Taxa de compressão
    size_ini = path.getsize(imgpath)
    size_end = path.getsize(filename)
    print("taxa: ", 1.* size_ini / size_end)

    messageToImage(imgname ,message_decoded_ints)

    img = cv2.imread(imgname)

    cv2.imshow("Lena", img)
    
    cv2.waitKey(0)

ex6()



KeyError: '2'

## Bibliografia

- CSM SLIDES 2 codif huffman sv1920 disponibilizados no Moodle Isel 2019 na disciplina codificação de sinais multimédia.
